In [1]:
# 修改镜像放导入相关库的前面
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from unsloth import FastLanguageModel
from trl import SFTTrainer
from datasets import load_dataset
from typing import Any
from pydantic import BaseModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/media/a100/data/WQ/private/LLMTwin/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
class LoadModelConfig(BaseModel):
    max_seq_length: int = 2048
    dtype: Any | None = None # None for auto detection
    load_in_4bit: bool = False

class PeftModelConfig(BaseModel):
    r: int = 16
    target_modules: Any = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    lora_alpha: int = 16
    lora_dropout: float = 0
    bias: str = "none"

In [3]:
loadconfig = LoadModelConfig()
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Meta-Llama-3.1-8B',
    **loadconfig.model_dump(),
    )

==((====))==  Unsloth 2025.12.8: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 2. Max memory: 79.251 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.09it/s]


In [4]:
peftconfig = PeftModelConfig()
model = FastLanguageModel.get_peft_model(
    model=model,
    **peftconfig.model_dump(),
)

Unsloth 2025.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
dataset1 = load_dataset("mlabonne/llmtwin", split="train")
dataset2 = load_dataset("mlabonne/FineTome-Alpaca-100k", split="train[:10000]")

In [6]:
from datasets import concatenate_datasets
dataset = concatenate_datasets([dataset1, dataset2])

alpaca_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token
def format_samples_sft(examples):
    text = []
    for instruction, output in zip(examples["instruction"], examples["output"], strict=False):
        message = alpaca_template.format(instruction, output) + EOS_TOKEN
        text.append(message)
    return {"text": text}

dataset = dataset.map(format_samples_sft, batched=True, remove_columns=dataset.column_names)
# dataset = dataset.map(format_samples_sft, batched=True,)
dataset = dataset.train_test_split(test_size=0.05)

In [7]:
print(dataset['train'][0]['text'][:100])  # 应该是正常文本

Below is an instruction that describes a task. Write a response that appropriately completes the req


In [8]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(tokenizer, chat_template='chatml')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


In [37]:
from transformers import TrainingArguments


trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        dataset_text_field="text",
        max_seq_length=2048,
        packing=False,
        args=TrainingArguments(
            learning_rate=3e-4,
            logging_steps=1,
            report_to='tensorboard',
            seed=0,
        )
    )

In [ ]:
# trainer.train()

In [9]:
test_input = tokenizer(dataset['train'][0]['text'], return_tensors='pt').to(model.device)
import torch
with torch.no_grad():
    outputs = model(**test_input, labels=test_input.input_ids)
    print("Loss type:", type(outputs.loss))
    print("Loss value:", outputs.loss)

Loss type: <class 'torch.Tensor'>
Loss value: tensor(1.2383, device='cuda:1')


In [17]:
loss = outputs["loss"]
print(loss)
loss*=2
print(loss)
loss.mean()

tensor(4.9531, device='cuda:1')
tensor(9.9062, device='cuda:1')


tensor(9.9062, device='cuda:1')

In [ ]:
def finetune(
        model_name:str, 
        loadconfig: "LoadModelConfig",
        peftconfig: "PeftModelConfig",
        ):
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        **loadconfig.model_dump(),
        )
    
    model = FastLanguageModel.get_peft_model(
        model=model,
        **peftconfig.model_dump(),
    )

    train_dataset = load_dataset('mlabonne/FineTome-Alpaca-100k')

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
    )

